In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION=os.getenv("AZURE_OPENAI_API_VERSION")

In [2]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
# If you encounter an error here are some ways to troubleshoot:
# - you can exclude the problematic credential by using a parameter (ex. exclude_shared_token_cache_credential=True)
# - For your credential, grant Cognitive Services User and Cognitive Services OpenAI User roles - EVEN IF YOU HAVE OWNER/ADMIN RIGHTS.
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_version=AZURE_OPENAI_API_VERSION,
  azure_ad_token_provider=token_provider
)

In [3]:
IMAGE_PATH="../media/image-whiteboard-stickynotes.png"

In [4]:
import base64

def executeGptVision(encoded_image, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    }
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content

In [5]:
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
noteType = executeGptVision(encoded_image, system_prompt_file="../prompts/detectNoteType.txt")
print (noteType)

WHITEBOARD


In [6]:
if noteType == "PAPER":
    ocrFile = "../prompts/ocrPaper.txt"
elif noteType == "WHITEBOARD":
    ocrFile = "../prompts/ocrWhiteboard.txt"
else:
    ocrFile = "../prompts/ocrImage.txt"

extractedText = executeGptVision(encoded_image, system_prompt_file=ocrFile)
print (extractedText)

# Help Us Solve These Problems!

## It's too cold in the Library
- Make some areas warmer, or colder than other rooms
- Space heaters/blankets
- Bring a sweater
- Have indoor plants
- Small heaters in study rooms
- Blanket station
- Library can provide blankets
- Bring a sweater
- Extra layers
- Partner with local sports teams to sell sports sweaters
- Bring a sweater
- Cozy sweaters in the library
- Snuggle
- Self-heating blankets
- Bring a sweater

## People feel uncomfortable/unsafe napping in the Library
- Have designated nap areas (like Google)
- Secure your bag
- Nap on a bean bag or make a nook to hide in
- Link your backpack
- Bring a friend
- Bring a blanket
- Take a nap at home
- Give staff a trusted moderator and drop corner
- Have lockers
- Napping pods
- Sleep bags
- Wear socks
- Bring a blanket
- Read and nap after class
- Quiet area dedicated to napping
- Study rooms

## People can walk into my reserved study room - if I'm there or not
- Plan ahead of time
- Box fort - m

In [7]:
def executeCompletion(text, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ]

    response = client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content

In [8]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(text=extractedText, system_prompt_file="../prompts/proofread.txt")
    print (extractedText)

# Help Us Solve These Problems!

## It's too cold in the Library
- Make some areas warmer or colder than other rooms
- Space heaters/blankets
- Bring a sweater
- Have indoor plants
- Small heaters in study rooms
- Blanket station
- Library can provide blankets
- Bring a sweater
- Extra layers
- Partner with local sports teams to sell sports sweaters
- Bring a sweater
- Cozy sweaters in the library
- Snuggle
- Self-heating blankets
- Bring a sweater

## People feel uncomfortable/unsafe napping in the Library
- Have designated nap areas (like Google)
- Secure your bag
- Nap on a bean bag or make a nook to hide in
- Link your backpack
- Bring a friend
- Bring a blanket
- Take a nap at home
- Give staff a trusted moderator and drop corner
- Have lockers
- Napping pods
- Sleep bags
- Wear socks
- Bring a blanket
- Read and nap after class
- Quiet area dedicated to napping
- Study rooms

## People can walk into my reserved study room - if I'm there or not
- Plan ahead of time
- Box fort - ma

In [9]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(extractedText, system_prompt_file="../prompts/sectionHeader.txt")
    print (extractedText)

# Help Us Solve These Problems!

## Temperature Solutions
- Make some areas warmer or colder than other rooms
- Space heaters/blankets
- Bring a sweater
- Have indoor plants
- Small heaters in study rooms
- Blanket station
- Library can provide blankets
- Bring a sweater
- Extra layers
- Partner with local sports teams to sell sports sweaters
- Bring a sweater
- Cozy sweaters in the library
- Snuggle
- Self-heating blankets
- Bring a sweater

## Napping Comfort
- Have designated nap areas (like Google)
- Secure your bag
- Nap on a bean bag or make a nook to hide in
- Link your backpack
- Bring a friend
- Bring a blanket
- Take a nap at home
- Give staff a trusted moderator and drop corner
- Have lockers
- Napping pods
- Sleep bags
- Wear socks
- Bring a blanket
- Read and nap after class
- Quiet area dedicated to napping
- Study rooms

## Study Room Security
- Plan ahead of time
- Box fort - make your own!
- Have a small section of locked rooms
- Lockers in study rooms
- Whiteboards on

In [10]:
import os
import datetime

filename = os.path.basename(IMAGE_PATH)
last_updated = datetime.datetime.fromtimestamp(os.path.getmtime(IMAGE_PATH)).strftime('%Y-%m-%d %H:%M')

result = f"""---
note-type: {noteType}
created-date: {last_updated}
last-updated: {last_updated}
---
# {filename}
![{filename}]({IMAGE_PATH})
{extractedText}"""

# save result to output.md
with open("output.md", "w") as file:
    file.write(result)

print(result)


---
note-type: WHITEBOARD
created-date: 2024-06-17 18:08
last-updated: 2024-06-17 18:08
---
# image-whiteboard-stickynotes.png
![image-whiteboard-stickynotes.png](../media/image-whiteboard-stickynotes.png)
# Help Us Solve These Problems!

## Temperature Solutions
- Make some areas warmer or colder than other rooms
- Space heaters/blankets
- Bring a sweater
- Have indoor plants
- Small heaters in study rooms
- Blanket station
- Library can provide blankets
- Bring a sweater
- Extra layers
- Partner with local sports teams to sell sports sweaters
- Bring a sweater
- Cozy sweaters in the library
- Snuggle
- Self-heating blankets
- Bring a sweater

## Napping Comfort
- Have designated nap areas (like Google)
- Secure your bag
- Nap on a bean bag or make a nook to hide in
- Link your backpack
- Bring a friend
- Bring a blanket
- Take a nap at home
- Give staff a trusted moderator and drop corner
- Have lockers
- Napping pods
- Sleep bags
- Wear socks
- Bring a blanket
- Read and nap after c